# **Autoencoder**

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

from PIL import Image

import torch
from torch.utils.data import DataLoader

from src.autoencoder.autoencoder import ConvAutoencoder
from src.data.dataset import get_split_image_files, SatelliteImageDataset
from src.utils.trainer import Trainer

DATA_PATH = '../data/'
MAXAR_PRE_PATH = DATA_PATH + 'maxar/pre/'

BATCH_SIZE = 32
ACCUMULATION_STEPS = 1
LEARNING_RATE = 1e-3
EPOCHS = 1
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

Image.MAX_IMAGE_PIXELS = 303038464

In [3]:
train_image_files, test_image_files, val_image_files = get_split_image_files(MAXAR_PRE_PATH)

train_dataset = SatelliteImageDataset(image_files=train_image_files)
test_dataset = SatelliteImageDataset(image_files=test_image_files)
val_dataset = SatelliteImageDataset(image_files=val_image_files)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [4]:
encoder_channels = [3, 4, 16, 32, 64]
decoder_channels = [64, 32, 16, 4, 3]
autoencoder = ConvAutoencoder(encoder_channels, decoder_channels).to(DEVICE)

In [5]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=LEARNING_RATE)
trainer = Trainer(
    model=autoencoder, 
    device=DEVICE,
    criterion=criterion,
    accumulation_steps=ACCUMULATION_STEPS,
    print_statistics=True
)
statistics = trainer.train_autoencoder(
    train_loader=train_dataloader, 
    val_loader=val_dataloader,
    optimizer=optimizer,
    num_epochs=EPOCHS,
)

🚀 Training autoencoder model for 1 epochs...


Epochs:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 